In [1]:
import cv2
import os
import numpy as np # importing the necessary libraries

In [2]:
face_cascade=cv2.CascadeClassifier(r'D:\Others\Projects\LBPH\haarcascade_frontalface_default.xml')
# creating a face detector
recognizer=cv2.face.LBPHFaceRecognizer_create() # generating face recognition model

In [4]:
def capture_images(User): # defining function to capture and store face images
    if not os.path.exists('Faces'):
        os.makedirs('Faces') # creating a new dictionary for save captured images
    cap=cv2.VideoCapture(0)
    
    count=0
    while True:
        ret,frame=cap.read()
        gray=cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY) # converting to gray frame
        faces=face_cascade.detectMultiScale(gray,scaleFactor=1.1,minNeighbors=5,minSize=(30,30)) # detecting faces in gray frame

        for (x,y,w,h) in faces:
            cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,0),2)
            cv2.imwrite(f'Faces/{User}_{count}.jpg',gray[y:y+h,x:x+w]) # store the captures images in the created folder
            count=count+1
        cv2.imshow('capture face',frame)
        if cv2.waitKey(1) & 0xFF==ord('q'):
            break
        if count>=400:
            break
    cap.release()
    cv2.destroyAllWindows()

In [25]:
capture_images('Athiresh') # creating data sets of user faces 

In [36]:
capture_images('Hailee')

In [37]:
capture_images('Neymar')

In [5]:
capture_images('Yadhu')

In [ ]:
#capture_images('New user') to add more user faces datasets

In [6]:
label={'Athiresh':0,'Hailee':1,'Neymar':2,'Yadhu':3} # creating a dictionary of respective usernames to encode into the numerical vector
label

{'Athiresh': 0, 'Hailee': 1, 'Neymar': 2, 'Yadhu': 4}

In [8]:
def train_model(label): # defining function to train model
    faces=[]
    labels=[] # creating lists to store the face samples and their corresponding labels

    for file_name in os.listdir('Faces'):
        name=file_name.split('_')[0] # extracting the username from file name

        image=cv2.imread(os.path.join('Faces',file_name))
        gray=cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)

        detected_faces=face_cascade.detectMultiScale(gray,scaleFactor=1.1,minNeighbors=5,minSize=(30,30))

        if len(detected_faces)>0: # cheching if face is detected and cropping the face region

            face_crop=gray[detected_faces[0][1]:detected_faces[0][1]+detected_faces[0][3],
                           detected_faces[0][0]:detected_faces[0][0]+detected_faces[0][2]]
            faces.append(face_crop)
            labels.append(label[name]) # faces and labels list appending

    recognizer=cv2.face.LBPHFaceRecognizer_create() # training the face recognition model using the faces and labels
    recognizer.train(faces,np.array(labels))
    return recognizer

Recognizer=train_model(label)
Recognizer

< cv2.face.LBPHFaceRecognizer 000001C033907D30>

In [9]:
def recognize_faces(recognizer,label): # function to recognize faces
    cap=cv2.VideoCapture(0)
    label_name={value:key for key, value in label.items()}
    while True:
        ret,frame=cap.read()
        gray=cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
        faces=face_cascade.detectMultiScale(gray,scaleFactor=1.1,minNeighbors=5,minSize=(30,30))

        for (x,y,w,h) in faces:

            label,confidence=recognizer.predict(gray[y:y+h, x:x+w]) # recognize the face using trained model
            if confidence>90:
                cv2.putText(frame,'Unknown',(x,y-10),cv2.FONT_HERSHEY_SIMPLEX,1,(255,0,0),2) # frame with unknown face
                cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,0),2)
            
            else:
                cv2.putText(frame,label_name[label],(x,y-10),cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,255),2) # frame with face and label name
                cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,0),2)

        cv2.imshow('recognize_faces',frame)
        if cv2.waitKey(1) & 0xFF==ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

In [20]:
recognize_faces(Recognizer,label) # real time recognition